In [ ]:
# Take all JSON from Blob Container and upload to Azure Search

In [77]:
import globals

import os
import pickle

import json
import requests
from pprint import pprint

from nltk.tokenize import sent_tokenize, word_tokenize

import time

In [2]:
max_sentences = 25

In [59]:
def processFile(data):
    try:
        json_response = requests.post(url = globals.ta_base_url, json= data).json()
        return json_response
    except Exception as ex:
        return {"error": [str(ex)]}


def createSearchUploadDoc(docID, json_content):
    try:
               
        CONDITION_QUALIFIER = []
        DIAGNOSIS = []
        DIRECTION = []
        EXAMINATION_NAME = []
        EXAMINATION_RELATION = []
        FAMILY_RELATION = []
        GENDER = []
        GENE = []
        MEDICATION_CLASS = []
        MEDICATION_NAME = []
        ROUTE_OR_MODE = []
        SYMPTOM_OR_SIGN = []
        TREATMENT_NAME = []
        BODY_STRUCTURE = []
        VARIANT = []

        
#         remove if negated....

        for doc in json_content['documents']:
            for ent in doc['entities']:
                if 'links' in ent:
                    for link in ent['links']:
                        if 'UMLS' in str(link):
                            if link['id'] in umls_concept_dict:
                                if ent['type'] == 'TREATMENT_NAME':
                                    TREATMENT_NAME.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'BODY_STRUCTURE':
                                    BODY_STRUCTURE.append(umls_concept_dict[link['id']])
                                if ent['type'] == 'CONDITION_QUALIFIER':
                                    CONDITION_QUALIFIER.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'DIAGNOSIS':
                                    DIAGNOSIS.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'DIRECTION':
                                    DIRECTION.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'EXAMINATION_NAME':
                                    EXAMINATION_NAME.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'EXAMINATION_RELATION':
                                    EXAMINATION_RELATION.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'FAMILY_RELATION':
                                    FAMILY_RELATION.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'GENDER':
                                    GENDER.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'GENE':
                                    GENE.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'MEDICATION_CLASS':
                                    MEDICATION_CLASS.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'MEDICATION_NAME':
                                    MEDICATION_NAME.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'ROUTE_OR_MODE':
                                    ROUTE_OR_MODE.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'SYMPTOM_OR_SIGN':
                                    SYMPTOM_OR_SIGN.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'TREATMENT_NAME':
                                    TREATMENT_NAME.append(umls_concept_dict[link['id']])
                                elif ent['type'] == 'VARIANT':
                                    VARIANT.append(umls_concept_dict[link['id']])

                                

        #unique terms and back to list
        BODY_STRUCTURE = list(set(BODY_STRUCTURE))
        CONDITION_QUALIFIER = list(set(CONDITION_QUALIFIER))
        DIAGNOSIS = list(set(DIAGNOSIS))
        DIRECTION = list(set(DIRECTION))
        EXAMINATION_NAME = list(set(EXAMINATION_NAME))
        EXAMINATION_RELATION = list(set(EXAMINATION_RELATION))
        FAMILY_RELATION = list(set(FAMILY_RELATION))
        GENDER = list(set(GENDER))
        GENE = list(set(GENE))
        MEDICATION_CLASS = list(set(MEDICATION_CLASS))
        MEDICATION_NAME = list(set(MEDICATION_NAME))
        ROUTE_OR_MODE = list(set(ROUTE_OR_MODE))
        SYMPTOM_OR_SIGN = list(set(SYMPTOM_OR_SIGN))
        TREATMENT_NAME = list(set(TREATMENT_NAME))
        VARIANT = list(set(VARIANT))

        return {"@search.action": "merge","docID": docID, "appliedMedicalEntities": True, "treatmentName":TREATMENT_NAME, "examinationName": EXAMINATION_NAME, 
                                   "bodyStructure": BODY_STRUCTURE, "diagnosis": DIAGNOSIS, "conditionQualifier": CONDITION_QUALIFIER, "direction": DIRECTION,
                                   "examinationRelation": EXAMINATION_RELATION, "familyRelation": FAMILY_RELATION, "gender": GENDER, "gene": GENE,
                                   "medicationClass": MEDICATION_CLASS, "medicationName": MEDICATION_NAME, "routeOrMode": ROUTE_OR_MODE, "symptomOrSign": SYMPTOM_OR_SIGN,
                                  "variant": VARIANT, "taHealthJSON": str(json_content)}
#         documents = {"value": []}
#         documents["value"].append({"@search.action": "merge","docID": docID, "treatmentName":TREATMENT_NAME, "examinationName": EXAMINATION_NAME, 
#                                    "bodyStructure": BODY_STRUCTURE, "diagnosis": DIAGNOSIS, "conditionQualifier": CONDITION_QUALIFIER, "direction": DIRECTION,
#                                    "examinationRelation": EXAMINATION_RELATION, "familyRelation": FAMILY_RELATION, "gender": GENDER, "gene": GENE,
#                                    "medicationClass": MEDICATION_CLASS, "medicationName": MEDICATION_NAME, "routeOrMode": ROUTE_OR_MODE, "symptomOrSign": SYMPTOM_OR_SIGN,
#                                   "variant": VARIANT})
#         url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
#         response  = requests.post(url, headers=globals.headers, json=documents)

    except Exception as ex:
        print ("Error:", ex)
        return {"@search.action": "merge","docID": docID}


In [4]:
# Load the UMLS dictionary of concept id's to names
with open(os.path.join(globals.umls_dir, 'umls_concept_dict.pickle'), 'rb') as handle:
    umls_concept_dict = pickle.load(handle)


In [60]:
def createTAHealthDoc(title, abstract, body):
    data = {"documents": []}

    try:
        id_counter = 0
        if len(title) > 1:
            id_counter += 1
            data["documents"].append({
                      "language": "en",
                      "id": str(id_counter),
                      "text": title
                    })

        id_counter += 1
        # Split abstract into sentences
        abstractBodySentences = sent_tokenize(abstract)

        for c in abstractBodySentences:
            data["documents"].append({
                  "language": "en",
                  "id": str(id_counter),
                  "text": c
                })
            id_counter += 1

        bodySentences = sent_tokenize(body)

        if id_counter <= max_sentences:
            for c in bodySentences:
                data["documents"].append({
                      "language": "en",
                      "id": str(id_counter),
                      "text": c
                    })
                id_counter += 1
                if id_counter == max_sentences:
                    break

    except Exception as ex:
            print ("Error:", ex)

    return data


In [76]:
print ("Uploading concepts to Azure Cognitive Search...")

# Get a document that has not been completed
url = globals.endpoint + "indexes/" + globals.indexName + "/docs" + globals.api_version + "&$count=true&$top=20&search=*&$select=docID,title,abstractContent,body&$filter=appliedMedicalEntities eq null"
response  = requests.get(url, headers=globals.headers)
docs = response.json()

while len(docs["value"]) > 0:
    print ('[' +  str(docs['@odata.count']) + " left]...")
    for doc in docs["value"]:
        docID = doc['docID']
        print ("Processing docID:", docID)

        ta_health_doc = createTAHealthDoc(doc["title"], doc["abstractContent"], doc["body"])

        ta_health_resp = processFile(ta_health_doc)
        if 'error' not in ta_health_resp:
            search_upload = createSearchUploadDoc(docID, ta_health_resp)

            documents = {"value": []}
            documents["value"].append(search_upload)
            url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
            upload_response  = requests.post(url, headers=globals.headers, json=documents)

            if upload_response.json()['value'][0]['status'] != True:
                print ("Error:", upload_response.json()['value'][0]['errorMessage'])
        else:
            print ('Error: ', ta_health_resp['error'])
            time.sleep(3)

            
    # Get another doc...
    time.sleep(3)
    url = globals.endpoint + "indexes/" + globals.indexName + "/docs" + globals.api_version + "&$count=true&$top=20&search=*&$select=docID,title,abstractContent,body&$filter=appliedMedicalEntities eq null"
    response  = requests.get(url, headers=globals.headers)
    docs = response.json()



Uploading concepts to Azure Cognitive Search...
[161 left]...
Processing docID: 0d90a077ea399c29a8b22105bd6a6bc61613579c
Processing docID: 0ff1d5f15854ee14f5a1739cc60c2d488d025b70
Processing docID: 13e81a52ac2f1dbb059eeab61f739d979ae3412d
Processing docID: 15c7e413de2e615378e1c0d4fa23d1f3425af285
Processing docID: 1c47f04cd1e7a5ffa80df67872b379342ed9c6f4
Processing docID: 1d93c5f7a7851227cc1aaf7ed0f4ee3aa72ac159
Processing docID: 1fa43de01deff122b935e7bb0677f82111ff867e
Processing docID: 238b1d30ab2a34817abf6b244d6ff4e84d1692d7
Processing docID: 26ad5d7fbb87bc58c2784d00df9d16840a5954f3
Processing docID: 2ae81644d62af97d80322750eaf154c025fe3ac8
[891 left]...
Processing docID: 453b25bc86ec94ab3326543c8ca885218b451312
Processing docID: 46f7f9afbe9a94ebecc0ee94525a059695886e58
Processing docID: 54a7f6c2c6e4b47a7856f5e4e22c7a83549e1c49
Processing docID: 54dad9350708facd9351fde49824cbcf3593e4ac
Processing docID: 563d2eb9eaa8c58e4ee88cfd706ba5a6b007e24d
Processing docID: 565f8ca61cbddeac8383a

KeyboardInterrupt: 